In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc
from prj.config import DATA_DIR


BASE_PATH = DATA_DIR / 'train'


partition=5
train_ds = pl.concat([
    pl.read_parquet(BASE_PATH / f'partition_id={i}' / 'part-0.parquet')
    for i in range(partition, partition + 1)
]).sort('date_id', 'time_id', 'symbol_id')
features = [col for col in train_ds.columns if col.startswith('feature_')]
target_feature = 'responder_6'


In [2]:
features = [f'feature_00', 'feature_01', 'feature_02']
features = [col for col in train_ds.columns if col.startswith('feature_')]
numerical_features = features
categorical_features = []
time_cols = ['date_id', 'time_id']

train_ds = train_ds.select(time_cols + ['symbol_id', 'weight'] + features + [target_feature])
train_ds.head(3)

date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_6
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
850,0,0,2.087724,-0.276877,-2.385324,-1.086325,0.049463,3.427029,-4.671824,0.054977,-0.259751,1.343003,11,7,76,-0.793587,2.523406,0.303231,null,0.523913,null,-1.567069,-0.965586,0.014156,-0.171976,1.015679,0.746074,-1.633316,-1.309486,0.965614,1.612443,0.82306,-0.027811,0.72484,-0.184198,null,…,0.050192,null,-0.954613,2.004981,-1.557791,0.678891,-0.066386,null,2.456588,null,null,-1.159385,null,-0.889724,1.428067,null,0.817551,0.299599,0.352903,-0.328996,-0.151735,-0.224472,-1.477134,-1.643559,-0.556531,2.815019,0.356358,-0.527251,1.609195,0.076337,null,null,-0.228297,-0.273781,-0.277999,-0.295312,1.461546
850,0,1,3.752097,-0.168178,-2.161023,-0.511679,0.192425,3.162096,-4.386098,0.130385,-0.368283,1.913416,11,7,76,-0.660111,3.052153,0.071869,null,0.001913,null,-0.625688,-1.11523,0.185483,0.019226,1.916643,0.710887,-1.102333,-0.981141,0.521467,1.665925,1.461316,-0.358575,0.058004,0.021168,null,…,1.311157,null,-0.749923,1.793136,-2.108881,1.227915,-0.146708,null,0.888707,null,null,-1.427895,null,-1.575317,0.556004,null,0.321817,0.406464,0.352903,-0.388503,-0.100457,-0.201082,-1.926849,-1.763679,-0.612577,1.61283,-0.051637,-0.97052,2.79455,0.353143,null,null,-0.157027,-0.163802,-0.277016,-0.444008,0.789595
850,0,2,1.225099,-0.520426,-1.718115,-0.817358,-0.270528,3.314825,-2.578923,0.1102,-0.20174,2.072351,81,2,59,-0.528026,3.354508,0.327966,null,-0.215615,null,-1.260532,-2.04301,-1.31462,-0.239955,0.017958,-0.27587,-0.705935,-0.782762,0.268385,1.391267,1.265022,-0.539895,-0.351402,-0.209022,null,…,-1.394171,null,-1.067848,0.734942,-2.05364,-1.888152,-0.688585,null,-0.588629,null,null,-2.212862,null,-2.015984,0.025982,null,-4.632971,-2.559358,0.352903,-0.316812,-0.264718,-0.248274,-1.383873,-2.433391,-0.728091,4.478824,0.497227,-0.449675,1.648489,-0.001233,null,null,-0.012737,-0.081892,-0.209053,-0.267447,-2.848316


In [3]:
from prj.utils import get_null_count
get_null_count(train_ds).head(3)

column,count,count (%)
str,u32,f64
"""feature_39""",375700,7.024793
"""feature_42""",375700,7.024793
"""feature_50""",375700,7.024793


In [ ]:
train_ds = train_ds.with_columns(
    pl.col(features).fill_nan(None).forward_fill(limit=100).over('symbol_id').name.keep()
)
get_null_count(train_ds).head(3)

column,count,count (%)
str,u32,f64
"""feature_21""",94764,1.771886
"""feature_26""",94764,1.771886
"""feature_27""",94764,1.771886


In [5]:
# Fill the other nulls
train_ds = train_ds.with_columns(
    pl.col(features).fill_null(0.)
)
get_null_count(train_ds).head(3)

column,count,count (%)
str,u32,f64
"""date_id""",0,0.0
"""time_id""",0,0.0
"""symbol_id""",0,0.0


In [ ]:
# from prj.utils import build_rolling_stats

# rolling_stats = build_rolling_stats(train_ds, cols=numerical_features, window=30)
# rolling_stats.head(3)

In [ ]:
# from prj.utils import moving_z_score_norm

# train_ds = moving_z_score_norm(train_ds, rolling_stats_df=rolling_stats, cols=numerical_features, clip_bound=None)
# train_ds.head(3)

In [6]:
X = train_ds.select(features).to_pandas()
y = train_ds[target_feature].to_numpy()
w = train_ds['weight'].to_numpy()

X_train = X.iloc[:int(0.8 * len(X))]
y_train = y[:int(0.8 * len(y))]
w_train = w[:int(0.8 * len(w))]
X_val = X.iloc[int(0.8 * len(X)):]
y_val = y[int(0.8 * len(y)):]
w_val = w[int(0.8 * len(w)):]

X_train.shape, y_train.shape, w_train.shape, X_val.shape, y_val.shape, w_val.shape

((4278560, 79), (4278560,), (4278560,), (1069640, 79), (1069640,), (1069640,))

In [7]:
from prj.model.nn.mlp import MLP

model = MLP(
    input_dim=X_train.shape[1:],
)

In [8]:
from keras import optimizers as tfko
from keras import metrics as tfkm
from prj.model.nn.losses import WeightedZeroMeanR2Loss 
optimizer = tfko.Adam(learning_rate=1e-4)
loss = WeightedZeroMeanR2Loss()
metrics = [tfkm.R2Score(), tfkm.MeanSquaredError()]

model.fit(
    X_train, 
    y_train,
    sample_weight=w_train,
    validation_data=(X_val, y_val, w_val),
    metrics=metrics,
    optimizer=optimizer, 
    loss=loss, 
    early_stopping_rounds=5,
    epochs=5
)

Training with early stopping patience 5
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 79)]              0         
                                                                 
 Dense0 (Dense)              (None, 128)               10240     
                                                                 
 BatchNormalization0 (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 Activation0 (Activation)    (None, 128)               0         
                                                                 
 Dropout0 (Dropout)          (None, 128)               0         
                                                                 
 OutputDense (Dense)         (None, 1)                 129       
                     

In [9]:
model.save('./model')

In [11]:
y_pred = model.predict(X_val, batch_size=256)

4179/4179 [==============================] - 2s 519us/step


In [12]:
from prj.metrics import weighted_r2


weighted_r2(y_val, y_pred, weights=w_val)

: 

In [ ]:
from prj.metrics import weighted_mae, weighted_mse, weighted_r2, weighted_rmse

def evaluate(X: np.ndarray, y: np.ndarray, weights: np.ndarray = None) -> float:
    y_pred = model.model.predict(X, batch_size=256)  
    return {
        'r2_w': weighted_r2(y, y_pred, weights=weights),
        'mae_w': weighted_mae(y, y_pred, weights=weights),
        'mse_w': weighted_mse(y, y_pred, weights=weights),
        'rmse_w': weighted_rmse(y, y_pred, weights=weights),
    }
    
evaluate(X_val, y_val, w_val)

4179/4179 [==============================] - 2s 369us/step


: 